In [1]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [121]:
from generate_input_planar_pose import GenerateInputPlanarPose

directory = os.path.expanduser('~/Documents/data/')

generator = GenerateInputPlanarPose([
    directory + 'pushing/cylinder-cube-1.db',
], test_files=[
#    directory + 'all-1/all-1.db',
], output_folder='gen-push/')

generator.percent_test_set = 0.2
generator.generateInput({
    'did_grasp_weight': 0.13,
    'force_rewrite': True,
    'size_cropped': (240, 240),
    'size_input': (752, 480),
    'size_output': (32, 32),
})

Reward Mean: 0.0351
Progress: 1000 of 2394
Progress: 2000 of 2394
Written training data to /home/berscheid/Documents/data/pushing/gen-push/train.csv
Written test data to /home/berscheid/Documents/data/pushing/gen-push/test.csv


In [166]:
from tensorflow_loader import DataLoader

test_loader = DataLoader(generator.test_output_filename, label_fields=['reward', 'action_direction'])
train_loader = DataLoader(generator.train_output_filename, label_fields=['reward', 'action_direction'], batch_size=128)

test_loader.labels[:, 0] = 0.5 * (test_loader.labels[:, 0] + 1)
train_loader.labels[:, 0] = 0.5 * (train_loader.labels[:, 0] + 1)

reward_abs_mean = abs(test_loader.labels[:, 0] - 0.5).mean()
print(reward_abs_mean)

print(test_loader.labels[:, 0].mean(), test_loader.labels[:, 0].std())

Length: 463
Reward Mean: 0.0414549364387061
Length: 1931
Reward Mean: 0.03356556036894687
0.04424783875434233
0.520727468219353 0.11166881158867335


In [175]:
from tensorflow_utils import get_augmentation, single_class_split, tf_accuracy, tf_precision, tf_recall, tf_f1

tf.reset_default_graph()
tf.set_random_seed(4)


train_dataset = tf.data.Dataset.from_generator(train_loader.nextBatch, (tf.float32, tf.float32), ((None, None, None, 1), (None, 2)))
test_dataset = tf.data.Dataset.from_generator(test_loader.entireBatch, (tf.float32, tf.float32))


handle = tf.placeholder(tf.string, shape=(), name='handle')
iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()
    
image, label = iterator.get_next()


def act(x): return tf.nn.leaky_relu(x, 0.2)
def reg(l1=0.0, l2=0.1): return tf.contrib.layers.l1_l2_regularizer(l1, l2)
    
    
training = tf.placeholder_with_default(False, (), name='training')
apply_dropout = tf.placeholder_with_default(training, (), name='apply_dropout')
image = tf.identity(image, name='image')
label = tf.identity(label, name='label')

    
x = tf.layers.conv2d(image, 32, (5, 5), strides=(2, 2), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.2, training=apply_dropout)
    
x = tf.layers.conv2d(x, 48, (5, 5), strides=(1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.3, training=apply_dropout)

x = tf.layers.conv2d(x, 64, (5, 5), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.4, training=apply_dropout)

x = tf.layers.conv2d(x, 142, (6, 6), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.4, training=apply_dropout)
    
x = tf.layers.conv2d(x, 128, (1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.dropout(x, 0.4, training=apply_dropout)
    
logits = tf.layers.conv2d(x, 2, (1, 1), bias_regularizer=reg(l2=0.05))
prob = tf.nn.sigmoid(logits, name='prob')


reward, reward_pred = single_class_split(label, prob[:, 0, 0])


weights = (0.2 + tf.abs(reward - 0.5)) / (0.2 + reward_abs_mean)
loss = tf.losses.mean_squared_error(labels=reward, predictions=reward_pred, weights=weights)
loss_reg = loss + tf.add_n(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
loss2 = tf.losses.mean_squared_error(labels=reward, predictions=reward_pred)

accuracy = tf_accuracy(tf.round(reward), reward_pred)
precision = tf_precision(tf.round(reward), reward_pred)
recall = tf_recall(tf.round(reward), reward_pred)
f1 = tf_f1(precision, recall, beta=0.5)


extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  # For batch normalization layers
with tf.control_dependencies(extra_update_ops):
    train = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(loss_reg, name='train')

In [176]:
from tensorflow_model import Model

model = Model(
    test_metrices=[loss, loss, accuracy, recall, loss2],
    inputs={'image': image, 'apply_dropout': apply_dropout},
    outputs={'prob': prob},
    model_input_path=generator.model_directory + 'model-2',
    model_output_path=generator.model_directory + 'model-2'
)

model.fit(
    train_iterator, train_loader.batches_per_epoch, test_iterator,
    epochs=5000,
    early_stopping_patience=3000,
    load=True,
    save=True,
    export=False,
)

INFO:tensorflow:Restoring parameters from /home/berscheid/Documents/data/pushing/models/model-2
Epoch	Time[s]	Train Accuracy	Test Loss	Accuracy	Precision	Recall		F1
   -1	0.00	-1.0000		0.0103		0.0103		0.5140		0.7173		0.0067
    0	0.46	0.0025		0.0103		0.0103		0.5076		0.7147		0.0068
    1	0.57	0.0014		0.0103		0.0103		0.5119		0.7138		0.0068
    2	0.69	0.0027		0.0104		0.0104		0.5140		0.7134		0.0068
    3	0.81	0.0038		0.0104		0.0104		0.5184		0.7131		0.0069
    4	0.93	0.0024		0.0104		0.0104		0.5162		0.7129		0.0069
    5	1.05	0.0028		0.0104		0.0104		0.5140		0.7128		0.0069
    6	1.17	0.0028		0.0104		0.0104		0.5119		0.7127		0.0069
    7	1.29	0.0028		0.0104		0.0104		0.5119		0.7132		0.0069
    8	1.42	0.0034		0.0104		0.0104		0.5140		0.7147		0.0069
    9	1.54	0.0019		0.0104		0.0104		0.5162		0.7158		0.0070
   10	1.66	0.0025		0.0104		0.0104		0.5140		0.7168		0.0070
   11	1.78	0.0021		0.0104		0.0104		0.5119		0.7173		0.0070
   12	1.89	0.0024		0.0104		0.0104		0.5162		0.7180		0.0070
   13	2.01	0.0027		0.0

KeyboardInterrupt: 